# Question One: Data Access (MySQL)

#### Your task is to construct a single SQL query that returns the following results: Summarized total sales, total quantity sold, and total profit (which can be calculated as total sales less cogs) by the week number, store id, product category name. It is important to note that in this business, the week begins on a Tuesday.

In [ ]:
SELECT store_id, cat_name, 
	WEEK(time_stamp - INTERVAL 2 DAY) AS week_number, 
    SUM(sales) AS total_sales, 
    SUM(cogs) AS total_cogs,
	SUM(units) AS total_quantity, 
    SUM(sales) - SUM(cogs) AS total_profit
FROM 
	Product_Detail PD, Order_Detail OD
WHERE 
	PD.product=OD.product
GROUP BY 
	week_number, store_id, cat_name;

#### Bonus Question: How would you use Python to access this data and perform these operations?

I would implement pyspark code and run the SQL query. 

# Question Two: Data Transformation (Python 3)

#### You are asked to create a dataframe matching this format, where the contents of the cells represent the count of occurrences of product attribute by customer.

In [79]:
import pandas as pd
import numpy as np

Create the dataframes:

In [94]:
data1={'product':[100, 101, 102], 'tags':[['chocolate', 'sprinkles'], ['chocolate', 'filled'], ['glazed']]}
Product_Attributes=pd.DataFrame(data1)

data2={'customer':['A', 'A', 'B', 'C', 'C', 'B', 'A', 'C'], 'product':[100, 101, 101, 100, 102, 101, 100, 102]}
Purchase_History=pd.DataFrame(data2)

merged_attributes=pd.merge(Product_Attributes, Purchase_History, how='inner', on='product')
merged_attributes

,product,tags,customer
0,100,"[chocolate, sprinkles]",A
1,100,"[chocolate, sprinkles]",C
2,100,"[chocolate, sprinkles]",A
3,101,"[chocolate, filled]",A
4,101,"[chocolate, filled]",B
5,101,"[chocolate, filled]",B
6,102,[glazed],C
7,102,[glazed],C


Create one column per attribute and count the occourence of product attributes per customer: 

In [84]:
for tag in merged_attributes['tags']:
    for attribute in tag:
        merged_attributes[attribute]=merged_attributes.tags.apply(lambda x: x.count(attribute))

count_attributes=merged_attributes.groupby('customer').sum()
count_attributes

,product,chocolate,sprinkles,filled,glazed
customer,,,,,
A,301,3,2,1,0
B,202,2,0,2,0
C,304,1,1,0,2


#### Bonus Question: If the two starting tables were in a relational database or Hadoop cluster and each had a 100 million rows, how might your approach change?

I would MapReduce fuctions to break the job across multiple machines (Spark would be faster, if available). 

# Question 3: Hypothesis testing (Python 3)

#### Each visitor to the site is randomly exposed to one of four different product images; visitors are then tracked to see which ones make a purchase or not. Based on the data provided, which version of the image should be selected to be presented to all visitors and why?

Build the data frame:

In [8]:
data3={'Image': ['A', 'B', 'C', 'D'],'visitors': [21,180, 250, 100], 'purchases': [3, 30, 50, 15] }
purchases=pd.DataFrame(data3)

Add a column for the number of customer that did not buy:

In [9]:
purchases['no_puchase'] = purchases['visitors'] - purchases['purchases']
purchases

,Image,visitors,purchases,no_puchase
0,A,21,3,18
1,B,180,30,150
2,C,250,50,200
3,D,100,15,85


Make a contingency table:

In [11]:
contingency_images=purchases.drop(['Image', 'visitors'], axis=1)
contingency_images_ary=np.array(contingency_images)
contingency_images_ary

array([[  3,  18],
       [ 30, 150],
       [ 50, 200],
       [ 15,  85]])

Chi squared test:

Null hypothesis(H0): The choice of image does not affect purchase.

Alternative hypothesis (HA): The choice of image affects purchase.

In [12]:
from scipy.stats import chi2_contingency

In [13]:
chi2, p, dof, expected = chi2_contingency(contingency_images_ary)
print (p)

0.6371433271168085


As p>0.05, we accept the null hypothesis. Being exposed to one image or another DOES NOT affect purchasese.

#### Bonus Question: How would your analysis change if the visitors and purchase counts numbered in the millions?

Build the data frame

In [14]:
purchases_mill=purchases.copy(deep=True)
purchases_mill['visitors']=purchases_mill['visitors']*1000000
purchases_mill['purchases']=purchases_mill['purchases']*1000000
purchases_mill['no_puchase']=purchases_mill['visitors']-purchases_mill['purchases']
purchases_mill

,Image,visitors,purchases,no_puchase
0,A,21000000,3000000,18000000
1,B,180000000,30000000,150000000
2,C,250000000,50000000,200000000
3,D,100000000,15000000,85000000


Make a contingency table:

In [16]:
contingency_images_mill=purchases_mill.drop(['Image', 'visitors'], axis=1)
contingency_images_ary_mill=np.array(contingency_images_mill)
contingency_images_ary_mill

array([[  3000000,  18000000],
       [ 30000000, 150000000],
       [ 50000000, 200000000],
       [ 15000000,  85000000]])

Chi squared test:

In [17]:
chi2, p, dof, expected = chi2_contingency(contingency_images_ary_mill)
print (p)

0.0


p<0.05, if visitors and purchase counts are numbered in millions, we reject the null hypothesis. Therefore, being exposed to one image or another, in this case DOES affect purchase.

# Question 4: Model Evaluation

#### Using the diagnostic output and correlation matrix below, determine what is wrong with this model and explain your findings.

Null Hypothesis(H0): B1=B=B3=B4=B5=B6=B7
Alternative Hypothesis(HA): At least one of the B's does not equal to 0

In our case, the p-value associated to the F statistic is 0.01. This means that we reject the null hypothesis, therefore at least one of the B's is not equal to 0. The B's represent the expected change in Y when X shifts by one unit. In our case, ONLY B5 has a significant p value (p<0.05), in this case 0.01. Therefore, X1, X2, X3, X4, X6, X7, X8 and X9 do not have any effect on Y, only X5 has an effect on Y.

Multiple Linear Regression Model: Y= Intercept +B1X1 +B2X2 +B3X3 +B4X4 +B5X5 +B6X6 +B7X7 +B8X8 + e

For Multiple Linear Regression the validity of our model depends on the following assumptions:

1. Linearity. 
To test the linear relationship between the dependent and independent variables we would have to see the data represented in a scatterplot.

2. Constant variance

3. Normality
To test for normality we would have to represent a quantile-quantile plot.

4. Independent errors
In order to test assumptions 2 and 4 we would have to represent a residual plot (scatterplot of the residual values versus the fitted values).

Assumptions 1-4 should be checked and confirmed for each predictor variable.

5. Multicolinearity:
Our predictor variables have to be uncorrelated. To test for this assumption we can inspect the correlation matrix. In our case, this assumption is violated as, 7 out of our 8 variables are correlated to each other, all except  X1, presenting correlation coefficients from 0.57-0.91, with positive or negative sign. 

Our R2 is decent (0.52) but still low. It expresses the amount of variability of the response variable explained by the predictor variables. The adjusted R2 is adjusted by the number of predictor variables, in our case 8. Adjusting for the 8  predictors drops the value of R2 to 0.4, indicating that increasing the number of predictors do not explain more variability of Y.

In summary, multiple linear regression model does not fit the data well, as at least one assumptions is violated, multicolinearity. To reduce multicolinearity we can reduce the number of vaiables, in our case, to X5 which is  the only variable that has a significant effect on Y and it is correlated with all the other predictors except with X1.

To improve linearity, we could try to transform our data taking logs, using Box Cox transformation or
squared root on X1 which is the only predictor that is not correlated to X5 and try to improve its linear relationship 
with the response variable.

In this case, due to the lack of effect of most of the predictors on the response variable, regularization with Ridge or Lasso will not improve our model.  I would try to use a tree based model such as random forest or boosted trees where multicolinearity is not a problem. 

# Question 5: Data Analysis

#### Evaluate ShinyFancyObjects and determine if it would be a good investment.

In [66]:
data4 = {
    'cohort':['Jan-16', 'Feb-16', 'Mar-16', 'Apr-16', 'May-16', 'Jun-16', 'Jul-16', 'Aug-16', 'Sep-16', 'Oct-16','Nov-16', 'Dec-16', 'Jan-17', 'Feb-17'],
    'month_0':[1098, 1358, 1257, 1587, 1758, 1624, 1541, 868, 1143, 1253, 1835, 2155, 2422, 2531],
    'month_1':[875, 1019, 892, 1174, 1213, 1137, 1002, 521, 629, 639, 899, 862, 848, 0],
    'month_2':[800, 996, 741, 924, 1000, 954, 874, 499, 451, 338, 241, 205, 0, 0],
    'month_3':[765, 826, 705, 838, 751, 754, 621, 400, 384, 255, 107, 0, 0, 0],
    'month_4':[552, 713, 648, 699, 605, 403, 322, 158, 205, 117, 0, 0, 0, 0],
    'month_5':[421, 524, 498, 354, 305, 248, 201, 118, 100,  0, 0, 0, 0, 0],
    'month_6':[403, 500, 395, 205, 165, 205, 168, 75,  0, 0, 0, 0, 0, 0],
    'month_7':[382, 495, 205, 158, 111, 165, 105,  0, 0, 0, 0, 0, 0, 0],
    'month_8':[367, 389, 168, 104, 90, 100, 0, 0, 0, 0, 0, 0, 0, 0],
    'month_9':[341, 301, 132, 79, 65, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'month_10':[295, 257, 121, 55, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'month_11':[244, 178, 84,  0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'month_12':[201, 133, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'month_13':[185, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
}

In [67]:
Retention=pd.DataFrame(data4)
Retention_rate=Retention.copy(deep=True)

In [68]:
for i in range(1,14):
    month_var = 'month_' + str(i)
    Retention_rate[month_var]=list(map(lambda x,y: x*100/y, Retention_rate[month_var], Retention_rate['month_0']))

In [69]:
Retention_rate

,cohort,month_0,month_1,month_2,month_3,month_4,month_5,month_6,month_7,month_8,month_9,month_10,month_11,month_12,month_13
0,Jan-16,1098,79.690346,72.859745,69.672131,50.273224,38.342441,36.703097,34.790528,33.424408,31.056466,26.867031,22.222222,18.306011,16.848816
1,Feb-16,1358,75.036819,73.343152,60.824742,52.503682,38.586156,36.818851,36.450663,28.645066,22.164948,18.924890,13.107511,9.793814,0.000000
2,Mar-16,1257,70.962609,58.949881,56.085919,51.551313,39.618138,31.424025,16.308671,13.365155,10.501193,9.626094,6.682578,0.000000,0.000000
3,Apr-16,1587,73.976055,58.223062,52.804033,44.045369,22.306238,12.917454,9.955892,6.553245,4.977946,3.465658,0.000000,0.000000,0.000000
4,May-16,1758,68.998862,56.882821,42.718999,34.414107,17.349261,9.385666,6.313993,5.119454,3.697383,0.000000,0.000000,0.000000,0.000000
5,Jun-16,1624,70.012315,58.743842,46.428571,24.815271,15.270936,12.623153,10.160099,6.157635,0.000000,0.000000,0.000000,0.000000,0.000000
6,Jul-16,1541,65.022713,56.716418,40.298507,20.895522,13.043478,10.902012,6.813757,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Aug-16,868,60.023041,57.488479,46.082949,18.202765,13.594470,8.640553,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Sep-16,1143,55.030621,39.457568,33.595801,17.935258,8.748906,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Oct-16,1253,50.997606,26.975259,20.351157,9.337590,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


The Retention Rate table shows the percentage of customers still active relative to month 0, which is 100%. As we can see, ShinyFancyObject, although growing the customer number over the months, the retention rate of new customers is decreasing from January 2016 to February 2017, as can be seen comparing the new feature released in November 2016, even though increased the number of new customers, it did not improve the retention rate, which in month 3, went from 20.35% in October 2016 to 5.83% in November 2016. The November 16 two month retention is worse than ten month in April of the same year.

The new feature in November plus some unknown event in September is leading to much worse customer retention rates. Given the declining user retention rates I would not invest in the company until the issues leading to retention rates were identified, fixed, and numbers improved.

#### Bonus Question: What other data sources would be helpful in making your recommendation?

Another data souce that could be helpful is the average revenue per customer, to see if a possible increase in revenue per customer can compensate the decrease in retention rate.